# Import

In [4]:
%load_ext autoreload
%autoreload 2

In [27]:
# Machine leanrnign algo
from sklearn import tree

# data manipulation
import pandas as pd
import numpy as np

#visualize
import matplotlib.pyplot as plt
import seaborn


#
import os

# Helper functions

In [24]:
def split(x,n=None):
    if n==None:
        n=int(len(x)*.8)
    
    return x[:n],x[n:]

def split_data(df=train):
    y = df['TARGET'].copy()
    x = df.drop('TARGET',axis=1)
    global x_train,x_valid,y_train,y_valid
    
    x_train ,x_valid = split(x)
    y_train ,y_valid = split(x)

# Import data

In [7]:
os.listdir('../data')

['test.csv', 'train.csv', 'sample_submission.csv']

In [8]:
train = pd.read_csv('../data/train.csv')

In [11]:
test = pd.read_csv('../data/test.csv')

In [12]:
train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [25]:
split_data()

# Base line